In [1]:
import pandas as pd
from nba_api.stats.static import players
from nba_api.stats.endpoints import playercareerstats
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [2]:
#My Table:
class Kobe(Base):
    __tablename__ = 'kobe'
    id = Column(Integer, primary_key=True)
    player_name = Column(String)
    year = Column(Float)
    team = Column(String)
    ft = Column(Float)
    fg = Column(Float)
    assist = Column(Float)
    block = Column(Float)
    point = Column(Float)

# Table for you to load your summary. You can define the items you want.
class Jordan(Base):
    __tablename__ = 'jordan'
    id = Column(Integer, primary_key=True)
    player_name = Column(String)
    year = Column(Float)
    team = Column(String)
    ft = Column(Float)
    fg = Column(Float)
    assist = Column(Float)
    block = Column(Float)
    point = Column(Float)


In [4]:
engine = create_engine('sqlite:///ETL_DB.sqlite')

In [5]:
Base.metadata.create_all(engine)

In [6]:
from sqlalchemy.orm import Session
session = Session(engine)

In [7]:
csv_file = "Seasons_Stats.csv"
player_data_df = pd.read_csv(csv_file)
kb_overall_data=player_data_df.loc[player_data_df["Player"]=="Kobe Bryant",:]

In [8]:
kb_specific=kb_overall_data[["Player","Year","Tm","FT%","FG%","AST","BLK","PTS"]]
kb_specific=kb_specific.reset_index(drop=True)

In [9]:
kb_specific=kb_specific.rename(columns={"Player":"player_name","Year":"year","Tm":"team",
                                        "FT%":"ft","FG%":"fg","AST":"assist","BLK":"block","PTS":"point"})
kb_specific.head()

,player_name,year,team,ft,fg,assist,block,point
0,Kobe Bryant,1997.0,LAL,0.819,0.417,91.0,23.0,539.0
1,Kobe Bryant,1998.0,LAL,0.794,0.428,199.0,40.0,1220.0
2,Kobe Bryant,1999.0,LAL,0.839,0.465,190.0,50.0,996.0
3,Kobe Bryant,2000.0,LAL,0.821,0.468,323.0,62.0,1485.0
4,Kobe Bryant,2001.0,LAL,0.853,0.464,338.0,43.0,1938.0


In [10]:
nba_players = players.get_players()
for nba_player in nba_players:
    if nba_player['full_name']=='Michael Jordan':
        player_id = nba_player['id']

career = playercareerstats.PlayerCareerStats(player_id=str(player_id))
mj_career = career.get_data_frames()[0]
mj_career['player_name']='Michael Jordan'
mj_career=mj_career.rename(columns={"SEASON_ID":"year","TEAM_ABBREVIATION":"team",
                                        "FT_PCT":"ft","FG_PCT":"fg","AST":"assist","BLK":"block","PTS":"point"})
mj_career=mj_career[["player_name","year","team","ft","fg","assist","block","point"]]
mj_career.head()

,player_name,year,team,ft,fg,assist,block,point
0,Michael Jordan,1984-85,CHI,0.845,0.515,481,69,2313
1,Michael Jordan,1985-86,CHI,0.840,0.457,53,21,408
2,Michael Jordan,1986-87,CHI,0.857,0.482,377,125,3041
3,Michael Jordan,1987-88,CHI,0.841,0.535,485,131,2868
4,Michael Jordan,1988-89,CHI,0.850,0.538,650,65,2633


In [11]:
engine.table_names()

['jordan', 'kobe']

In [12]:
kb_specific.to_sql(name='kobe', con=engine, if_exists='append', index=False)

In [13]:
pd.read_sql_query('select * from kobe', con=engine).head()

,id,player_name,year,team,ft,fg,assist,block,point
0,1,Kobe Bryant,1997.0,LAL,0.819,0.417,91.0,23.0,539.0
1,2,Kobe Bryant,1998.0,LAL,0.794,0.428,199.0,40.0,1220.0
2,3,Kobe Bryant,1999.0,LAL,0.839,0.465,190.0,50.0,996.0
3,4,Kobe Bryant,2000.0,LAL,0.821,0.468,323.0,62.0,1485.0
4,5,Kobe Bryant,2001.0,LAL,0.853,0.464,338.0,43.0,1938.0


In [14]:
mj_career.to_sql(name='jordan', con=engine, if_exists='append', index=False)
pd.read_sql_query('select * from jordan', con=engine).head()

,id,player_name,year,team,ft,fg,assist,block,point
0,1,Michael Jordan,1984-85,CHI,0.845,0.515,481.0,69.0,2313.0
1,2,Michael Jordan,1985-86,CHI,0.840,0.457,53.0,21.0,408.0
2,3,Michael Jordan,1986-87,CHI,0.857,0.482,377.0,125.0,3041.0
3,4,Michael Jordan,1987-88,CHI,0.841,0.535,485.0,131.0,2868.0
4,5,Michael Jordan,1988-89,CHI,0.850,0.538,650.0,65.0,2633.0
